In [7]:
import numpy as np
import matplotlib.pyplot as plt


from Server import Server
from Customer import Customer
from Event_List import Event_List




In [8]:
def create_customer_Arrival(arrival_rate, max_time):
    '''
    Creates an Event_List object of customers' scheduled arrivals
    Inputs:
        arrival_rate - how fast customers arrive (using the exponential distribution)
        max_time - for how long customers arrive
    Outputs:
        Event_List of customers' scheduled arrivals
    '''
    np.random.seed(220)
    t = 0
    cust = 1
    arriv = 0
    customerlist = Event_List()
    customerlist.addEvent(arriv,Customer(arriv,cust),"Arrival")
    while arriv <= max_time:
        cust+=1
        rand_exp = np.random.exponential(arrival_rate,1)[0]
        arriv = rand_exp+t
        t+= rand_exp
        customerlist.addEvent(arriv,Customer(arriv,cust),"Arrival")

    return customerlist


In [9]:
def is_available(server_lst):
    """
    Takes in a list of servers and outputs a bool determining whether a server is available and/or the index of that server in the list
    input: server_lst- a list of server objects
    output:  [False ] or [True, index of server]

    """

    for i, server in enumerate(server_lst):
        if not server.isBusy():
            return [True, i]
    else:
        return [False]



In [10]:
def find_customer_server(server_lst,customer):
    '''
    Takes in a list of server objects and a customer object
    input: server_lst - list of server objects
           customer - customer object
    output: Server object that contains the server

    '''


    for server in server_lst:
        if customer.number == server.customer_number:
            return server


In [11]:
def simulateShop(filename, num_servers, server_rates):
    '''
    Writes a simulation of a queue to specified outfile
    given a specified number of servers and corresponding
    server rates.
    Inputs:
        filename - string that you want to output file to be called
        num_servers - number of servers in the "shop"
        server_rates - service rates for the servers
    Outputs:
        No output, writes simulation to file
    '''
    outfile = open(filename+".txt","w+")
    Queue = Event_List()
    customer_event = create_customer_Arrival(5,100)

    server_lst = []
    for x in range(num_servers):
        server_lst.append(Server(x+1,server_rates[x]))
    while len(customer_event):

        event_time, customer, event = customer_event.next()

        if event == "Arrival":
            available = is_available(sorted(server_lst))

            if available[0]:
                server = server_lst[available[1]]
                end_time = server.serve(customer,event_time)
                customer_event.addEvent(end_time,customer,"Departure")
            elif not available[0]:
                Queue.addEvent(event_time , customer, event)
        elif event == "Departure":
            server = find_customer_server(server_lst,customer)
            if server is not None:
                server.freeUp()
                if len(Queue):
                    event_time , customer, event = Queue.next()
                    available = is_available(sorted(server_lst))
                    if available[0]:
                        server = server_lst[available[1]]
                        end_time = server.serve(customer,event_time)
                        customer_event.addEvent(end_time,customer,"Departure")
        print(customer_event, file= outfile)
        for x in server_lst:
            print(x, file = outfile)
        print("=====Customer Line ===============\n", file = outfile)
        if event_time >=100:
            break

    outfile.close()


In [12]:
# Main script part
filename = input("Outfile name?")
num_servers = int(input("How many servers do you want?"))
server_rates = [0 for i in range(num_servers)]
for i in range(num_servers):
    server_rates[i] = int(input(f"What rate do you want server {i+1} to work at?"))
print("Queue simulation created as a file!")
simulateShop(filename, num_servers, server_rates)

Queue simulation created as a file!


NameError: name 'np' is not defined